In [2]:
!wget https://huggingface.co/omeryentur/phi-3-sql/resolve/main/phi-3-sql.Q4_K_M.gguf

--2024-06-20 08:15:50--  https://huggingface.co/omeryentur/phi-3-sql/resolve/main/phi-3-sql.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 3.163.189.90, 3.163.189.74, 3.163.189.114, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/56/6f/566fbe9d66384fd0ddaf4103740612ffa7c8223f34c3c98513c1437cabbb29d2/622694786f2d047424df9084cd03d9146c7a538f8320c9938a4d7a8da98adf2d?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27phi-3-sql.Q4_K_M.gguf%3B+filename%3D%22phi-3-sql.Q4_K_M.gguf%22%3B&Expires=1719130550&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxOTEzMDU1MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzU2LzZmLzU2NmZiZTlkNjYzODRmZDBkZGFmNDEwMzc0MDYxMmZmYTdjODIyM2YzNGMzYzk4NTEzYzE0MzdjYWJiYjI5ZDIvNjIyNjk0Nzg2ZjJkMDQ3NDI0ZGY5MDg0Y2QwM2Q5MTQ2YzdhNTM4ZjgzMjBjOTkzOGE0Z

In [26]:
!pip install --upgrade --quiet  transformers --quiet

In [4]:
from langchain_community.llms import LlamaCpp


In [5]:
client=LlamaCpp(model_path="/content/phi-3-sql.Q4_K_M.gguf")

llama_model_loader: loaded meta data with 25 key-value pairs and 291 tensors from /content/phi-3-sql.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = phi-3-sql
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 32064
llama_model_loader: - kv   3:                       llama.context_length u32              = 4096
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 3072
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  llama.feed_forward_length u32              = 8192
llama_model_loader: - kv   7:                 llama.rope.dimension_count u32  

In [6]:
from langchain.prompts.prompt import PromptTemplate


In [7]:
from langchain.sql_database import SQLDatabase


In [10]:
db_path="sqlite:///example.db"
db=SQLDatabase.from_uri(database_uri=db_path)

In [11]:
db.get_table_names()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `SQLDatabase.get_table_names` was deprecated in langchain-community 0.0.1 and will be removed in 0.3.0. Use get_usable_table_names instead.
  warn_deprecated(


['courses', 'students']

In [12]:
db._sample_rows_in_table_info=0

In [14]:
table_info=db.get_table_info()
table_info

'\nCREATE TABLE courses (\n\tid INTEGER, \n\tname TEXT NOT NULL, \n\tdescription TEXT, \n\tPRIMARY KEY (id)\n)\n\n\nCREATE TABLE students (\n\tid INTEGER, \n\tname TEXT NOT NULL, \n\tage INTEGER NOT NULL, \n\tPRIMARY KEY (id)\n)'

In [15]:
template="""
<|system|>
{table_info}

<|user|>
{question}
<|sql|>
"""


In [16]:
from langchain.schema import HumanMessage


In [22]:
question="Courses containing Introduction"

prompt=PromptTemplate.from_template(template)



In [23]:
res=client(prompt.format(table_info=table_info,question=question))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1442.77 ms
llama_print_timings:      sample time =       8.83 ms /    17 runs   (    0.52 ms per token,  1925.91 tokens per second)
llama_print_timings: prompt eval time =    2149.32 ms /    11 tokens (  195.39 ms per token,     5.12 tokens per second)
llama_print_timings:        eval time =    5708.12 ms /    16 runs   (  356.76 ms per token,     2.80 tokens per second)
llama_print_timings:       total time =    7880.77 ms /    27 tokens


In [24]:
sql_query=res
sql_query

'SELECT * FROM "courses" WHERE "description" LIKE \'%Introduction%\'\n'

In [25]:
db.run(sql_query)

"[(2, 'Physics', 'Introduction to Physics'), (4, 'Biology', 'Introduction to Biology'), (6, 'Geography', 'Introduction to Geography'), (9, 'Computer Science', 'Introduction to Computer Science')]"

In [30]:
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [32]:
from langchain_core.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

In [34]:
gpu_llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    device=0,  # replace with device_map="auto" to use the accelerate library.
    pipeline_kwargs={"max_new_tokens": 100},
)

gpu_chain = prompt | gpu_llm

question = "What is electroencephalography?"

print(gpu_chain.invoke({"question": question}))

Question: What is electroencephalography?

Answer: Let's think step by step. One of the most interesting parts of the study is identifying the time at which EEG signals are produced at a particular moment. Each time an electrode is added to a brain, it will pass time between the added electrodes. I've been working on the subject of EEG for some years now, and an EEG comes into play in a lot of clinical settings. My approach to studying EEG signals over the course of a year is to use visual analogues for the whole EEG field. In a sense, we are
